In [ ]:
#manually mount the drive!
HOME="/content/drive/My Drive"

In [ ]:
#@title installs
%cd /content
!rm -rf ddsp_gm2
!git clone --single-branch --branch using_vocoder https://github.com/gianmarcohutter/ddsp_gm2
%cd ddsp_gm2
%tensorflow_version 2.x
!pip install -e /content/ddsp_gm2[data_preparation]
!pip install -e /content/ddsp_gm2[ddsp] 
!pip install mir_eval

In [ ]:
#for phonemes
!apt-get update
!apt-get install -y swig libpulse-dev
!swig -version
!pip3 install pocketsphinx
!pip3 list | grep pocketsphinx

In [ ]:
#@title imports
#dont know if this is needed?
import ddsp

'''
import time

from ddsp.training import (data, decoders, encoders, models, preprocessing, 
                           train_util, trainers)
from ddsp.colab.colab_utils import play, specplot, DEFAULT_SAMPLE_RATE
import gin
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
'''

#for phonemes
import itertools
import pocketsphinx
import os

In [ ]:
#@title create test folder
#the last one showing up in the list below is where its actually saved
%cd {HOME}

#make a new testx folder:
i=0
while(True):
    test_folder="test"
    i=i+1
    test_folder=test_folder+str(i)
    print(test_folder)
    var=![ -d {test_folder} ] && echo 'does exist'
    if(len(var)==0):
      !mkdir {test_folder}
      #%cd test_folder
      break

In [ ]:
# copy audio to the test folder
DRIVE_DIR=os.path.join(HOME,test_folder)

#copy the all songs from that folder into the drive_dir
AUDIO_STORAGE=SAVE_DIR = os.path.join(HOME,"vocoder_alternative_jack_spleeter/.")
!cp -r "$AUDIO_STORAGE" "$DRIVE_DIR"


In [ ]:
!pwd
print(DRIVE_DIR)
#going back to /content is crucial! otherwise it gets saved in drive and reused the next time the script is run
%cd /content/ 

In [ ]:
import glob
import os

AUDIO_DIR = 'data/audio'
AUDIO_FILEPATTERN = AUDIO_DIR + '/*'
!mkdir -p $AUDIO_DIR

if DRIVE_DIR:
  SAVE_DIR = os.path.join(DRIVE_DIR, 'GM-Voice')
else:
  SAVE_DIR = '/content/models/GM-Voice'
!mkdir -p "$SAVE_DIR"

if DRIVE_DIR:
  mp3_files = glob.glob(os.path.join(DRIVE_DIR, '*.mp3'))
  wav_files = glob.glob(os.path.join(DRIVE_DIR, '*.wav'))
  audio_files = mp3_files + wav_files

for fname in audio_files:
  target_name = os.path.join(AUDIO_DIR, 
                             os.path.basename(fname).replace(' ', '_'))
  print('Copying {} to {}'.format(fname, target_name))
  !cp "$fname" $target_name

In [ ]:
import glob
import os

TRAIN_TFRECORD = 'data/train.tfrecord'
TRAIN_TFRECORD_FILEPATTERN = TRAIN_TFRECORD + '*'

# Copy dataset from drive if dataset has already been created.
drive_data_dir = os.path.join(DRIVE_DIR, 'data') 
drive_dataset_files = glob.glob(drive_data_dir + '/*')

if DRIVE_DIR and len(drive_dataset_files) > 0:
  !cp "$drive_data_dir"/* data/
  print("did not prepare the tfrecord new")

else:
  # Make a new dataset.
  print("prepare the tfrecord new")
  if not glob.glob(AUDIO_FILEPATTERN):
    raise ValueError('No audio files found. Please use the previous cell to '
                    'upload.')

  !ddsp_prepare_tfrecord_phonemes \
    --input_audio_filepatterns=$AUDIO_FILEPATTERN \
    --output_tfrecord_path=$TRAIN_TFRECORD \
    --num_shards=10 \
    --alsologtostderr

  # Copy dataset to drive for safe-keeping.
  if DRIVE_DIR:
    !mkdir "$drive_data_dir"/
    print('Saving to {}'.format(drive_data_dir))
    !cp $TRAIN_TFRECORD_FILEPATTERN "$drive_data_dir"/

In [ ]:
# for Quantile normalization

from ddsp.colab import colab_utils
import ddsp.training

#not sure what provider i need to choose here
data_provider = ddsp.training.data.TFRecordProvider(TRAIN_TFRECORD_FILEPATTERN)
#data_provider = ddsp.training.data_phoneme.TFRecordProviderPhoneme(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_dataset(shuffle=False)
PICKLE_FILE_PATH = os.path.join(SAVE_DIR, 'dataset_statistics.pkl')

colab_utils.save_dataset_statistics(data_provider, PICKLE_FILE_PATH,batch_size=16)

In [ ]:
from ddsp.colab import colab_utils
import ddsp.training
from matplotlib import pyplot as plt
import numpy as np

data_provider = ddsp.training.data_phoneme.TFRecordProviderPhoneme(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_dataset(shuffle=False)

try:
  ex = next(iter(dataset))
except StopIteration:
  raise ValueError(
      'TFRecord contains no examples. Please try re-running the pipeline with '
      'different audio file(s).')

colab_utils.specplot(ex['audio'])
colab_utils.play(ex['audio'])

for key, value in ex.items():
    print (key)

colab_utils.specplot(ex['alternative_audio'])
colab_utils.play(ex['alternative_audio'])

f, ax = plt.subplots(4, 1, figsize=(14, 4))
x = np.linspace(0, 4.0, 1000)
ax[0].set_ylabel('loudness_db')
ax[0].plot(x, ex['loudness_db'])
ax[1].set_ylabel('F0_Hz')
ax[1].set_xlabel('seconds')
ax[1].plot(x, ex['f0_hz'])
ax[2].set_ylabel('F0_confidence')
ax[2].set_xlabel('seconds')
ax[2].plot(x, ex['f0_confidence'])
ax[3].set_ylabel('phonemes')
ax[3].plot(x,ex['phoneme'])


#training




In [ ]:
%reload_ext tensorboard
import tensorboard as tb
tb.notebook.start('--logdir "{}"'.format(SAVE_DIR))

In [ ]:
  #run the actual training
  !ddsp_run \
  --mode=train \
  --alsologtostderr \
  --save_dir="$SAVE_DIR" \
  --gin_file=models/ae_phoneme.gin \
  --gin_file=datasets/tfrecord_phoneme.gin \
  --gin_param="TFRecordProviderPhoneme.file_pattern='$TRAIN_TFRECORD_FILEPATTERN'" \
  --gin_param="batch_size=16" \
  --gin_param="train_util.train.num_steps=24000" \
  --gin_param="train_util.train.steps_per_save=2400" \
  --gin_param="trainers.Trainer.checkpoints_to_keep=10"

In [ ]:
from google.colab import output
def beep():
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

beep()

In [ ]:
from ddsp.colab.colab_utils import play, specplot
import ddsp.training
import gin
from matplotlib import pyplot as plt
import numpy as np

data_provider = ddsp.training.data_phoneme.TFRecordProviderPhoneme(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_batch(batch_size=1, shuffle=True)

try:
  batch = next(iter(dataset))
except OutOfRangeError:
  raise ValueError(
      'TFRecord contains no examples. Please try re-running the pipeline with '
      'different audio file(s).')

# Parse the gin config.
gin_file = os.path.join(SAVE_DIR, 'operative_config-0.gin')
gin.parse_config_file(gin_file)

# Load model
model = ddsp.training.models.Autoencoder()
model.restore(SAVE_DIR)

# Resynthesize audio.
audio_gen = model(batch, training=False)
audio = batch['audio']

print('Original Audio')
specplot(audio)
play(audio)

print('Resynthesis')
specplot(audio_gen)
play(audio_gen)

In [ ]:
from ddsp.colab import colab_utils
import tensorflow as tf
import os

CHECKPOINT_ZIP = 'GM-Voice.zip'
latest_checkpoint_fname = os.path.basename(tf.train.latest_checkpoint(SAVE_DIR))
!cd "$SAVE_DIR" && zip $CHECKPOINT_ZIP $latest_checkpoint_fname* operative_config-0.gin dataset_statistics.pkl
!cp "$SAVE_DIR/$CHECKPOINT_ZIP" ./
#colab_utils.download(CHECKPOINT_ZIP)